# Applying model to a video

## Include libraries and functions

In [1]:
%run Squat_common_v2.ipynb
%run functions_angle_calculation.ipynb
%run functions_video.ipynb

2024-05-21 12:14:37.165413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 12:14:37.277939: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 12:14:37.281256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-21 12:14:37.281265: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

TF version: 2.10.0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Playing a video

In [8]:
# this frame_process does nothing for now, just returning the frame

def frame_process(frame):
    return frame

In [9]:
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

## Add some text to the video

In [5]:
text_scale = 0.65

def putText(frame, text, color = (0, 255, 0), position_x = 50, position_y = 50):
    global text_scale
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_position = (position_x, position_y)
    text_color = color
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)   


In [12]:
def frame_process(frame):
    text = f"HAHAHA"
    putText(frame, text)
    return frame

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

### Add a counter and angles to the video

In [32]:
frame_count = 0
knee_angle = 0
hip_angle = 0

def frame_process(frame):
    global frame_count
    frame_count = frame_count + 1
    text = f"Frame: {frame_count}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    
    putText(frame, text)
    return frame

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process))
thread.start()


ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

## Change the FPS of the video

In [2]:
fps = 3
frame_count = 0

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process, fps))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

NameError: name 'frame_process' is not defined

## Calculate angle for each frame

In [6]:
frame_count = 0
squat_count = 0

def frame_process(frame):
    global frame_count, squat_count, squat_state

    frame_count = frame_count + 1

    # calculate and display angle
    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    frame = cv2_resize_with_pad(frame)

    text = f"Frame count: {frame_count}, Squat Count: {squat_count}, Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    
    putText(frame, text)

    return frame


# Run
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process, fps))
thread.start()



ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

## Count the squat

### Define some helper functions

In [ ]:
def reset_counter():
    global frame_count, squat_count, squat_state
    squat_count = 0
    squat_state = "standing"
    frame_count = 0


In [15]:
def count_squat(hip_angle, knee_angle, squat_count, squat_state):
    # Gesture judgement from angle of hip-joint and knee-joint
    # if gesture is from squat (squat == 1) change to stand (squat == 0), repetition add by 1

    if hip_angle < 135 and knee_angle < 120:
        squat_state = "squating"
    elif hip_angle>135 and knee_angle>120:
        if squat_state == "squating":
            squat_count += 1
        squat_state = "standing"

    return squat_count, squat_state   

### Test the function

#### Case 1
He was standing but the angle now less than 135 and 120, the new state become squating

In [16]:
count_squat(120, 110, 0, "standing")

(0, 'squating')

#### Case 2
He was squating but the angle now less than 135 and 120, the state is not changed

In [17]:
count_squat(120, 110, 0, "squating")

(0, 'squating')

#### Case 3
He was squating but the angle now bigger than 135 and 120
- The state is changed back standing
- The count become 1

In [18]:
count_squat(150, 150, 0, "squating")

(1, 'standing')

#### Case 4
He was squating but the angle now bigger than 135 and 120
- The state is changed back standing
- The count become 6

In [29]:
count_squat(150, 150, 5, "squating")

(6, 'standing')

### Integrate into our program

In [20]:
squat_state = "standing"

reset_counter()
    
def frame_process(frame):
    global frame_count, squat_count, squat_state

    frame_count = frame_count + 1
    
    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    frame = cv2_resize_with_pad(frame)

    squat_count, squat_state = count_squat(hip_angle, knee_angle, squat_count, squat_state)
 
    text = f"Frame count: {frame_count}, Squat Count: {squat_count}, Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    
    if squat_state == "squating":
        text_color = (0,255,0)
    else:
        text_color = (128, 192, 64)

    putText(frame, text, text_color)

    return frame


# Run
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process, fps))
thread.start()



ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

## Annotate the keypoints

### Define some helper functions

In [22]:
# Define the color (BGR) and size

def cv2_circle(frame, xy, text=None):
    
    color = (255, 0, 0)  # Blue color
    radius = 5
    thickness = -1  # Solid circle
    
    # Draw the point
    cv2.circle(frame, xy, radius, color, thickness)

    if text is not None:
        x,y = xy
        
        putText(frame, text, (255,0,0),  int(x * 1.1), y)


def absolute_xy(frame, point):
    x = point[1] * frame.shape[1]
    y = point[0] * frame.shape[0]
    return (int(x), int(y))
    

In [27]:
squat_state = "standing"

reset_counter()
    
def frame_process(frame):
    global frame_count, squat_count, squat_state

    frame_count = frame_count + 1
    
    movenet_frame = convert_to_movenet_format(frame)
    movenet_output = movenet(movenet_frame)
    keypoints_with_scores = reoganize_output(movenet_output)

    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    frame = cv2_resize_with_pad(frame)

    squat_count, squat_state = count_squat(hip_angle, knee_angle, squat_count, squat_state)
 
    text = f"Frame count: {frame_count}, Squat Count: {squat_count}, Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    
    if squat_state == "squating":
        text_color = (0,255,0)
    else:
        text_color = (128, 192, 64)

    putText(frame, text, text_color)

    for body_part in body_parts:    
        cv2_circle(frame, absolute_xy(frame, keypoints_with_scores['right'][body_part]), body_part)

    return frame


# Run
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, "../resource/Produce_2_direct.mp4", 0, frame_process, fps))

thread.start()



ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

## Use Camera as the source

Instead of specifying the filename in the second parameters of `view()`, put `0` because `0` is normally the default camera of the system

In [33]:
fps = 300

In [34]:
text_scale = 0.65

reset_counter()

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, 0, -1, frame_process, fps))

thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None